# Top Model Configurations

**Import Packages**
The following cell will import the required packages, and print the their current version, and indicate how many GPU's are connected to the system.

In [16]:
# Author: Luke Collins
# Date Created: 2021-09-28
# Date Modified: 2021-09-28
# Description: This file contains the code used to run the experiments for the top model configuration experiments.

import os
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, roc_curve, f1_score, auc
from sklearn.metrics import precision_score, recall_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.applications import Xception, xception
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Precision, Recall

print("Numpy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("Seaborn:", sns.__version__)
#print("Matplotlib:", matplotlib.pyplot.__version__)
#print("Scikit-learn version:", sklearn.__version__)
print("Tensorflow version:", tf.version)
print("Number of GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Numpy version: 1.23.5
Pandas version: 2.0.1
Seaborn: 0.12.2
Tensorflow version: <module 'tensorflow._api.v2.version' from '/home/luke/Development/deepfake-detection-v2/.venv/lib/python3.10/site-packages/tensorflow/_api/v2/version/__init__.py'>
Number of GPUs Available:  1


**Notebook Configuration Cell**  
The following cell should always be run directly after the packages are imported, this is where you will configure directories to suit your system if reproducing this experiment at home. Do not proceed if your cells output produces and `false` for any of the directories.

In [17]:
# Load Directories
train_directory = "/home/luke/datasets/train-5"
test_directory = "/home/luke/datasets/test-5"
experiment_base_directory = "./experiments/top-model-configuration/"

# Create directories if they don't exist
os.makedirs(experiment_base_directory, exist_ok=True)

# Check if directories exist
print("Train directory exists:", os.path.isdir(train_directory))
print("Test directory exists:", os.path.isdir(test_directory))
print("Experiment directory exists:", os.path.isdir(experiment_base_directory))

Train directory exists: True
Test directory exists: True
Experiment directory exists: True


**Dataset Creation Utilities**  
The following cell will define the functions required to create the training, validation and test datasets.

In [18]:
def create_train_val_datasets(
    input_dir,
    img_height=299,
    img_width=299,
    batch_size=32,
    augment_training_data=False
):
    """
    Creates train and validation datasets from images in the input directory.

    Args:
        input_dir (str): The path to the input directory containing 'real' and 'fake' subdirectories.
        output_dir (str): The directory where the TFRecord files will be saved. Default is 'data/datasets'.
        img_height (int): The height of the input images. Default is 299.
        img_width (int): The width of the input images. Default is 299.
        batch_size (int): The batch size for the data generators. Default is 32.
        save_dataset (bool): Whether to save the datasets as TFRecord files. Default is True.

    Returns:
        tf.data.Dataset, tf.data.Dataset: The train and validation datasets.
    """
    # Create ImageDataGenerator with validation split
    if augment_training_data:
        datagenTrain = ImageDataGenerator(
                    preprocessing_function=xception.preprocess_input,
                    rotation_range=20,  # Increased rotation range
                    width_shift_range=0.2,  # Increased shift range
                    height_shift_range=0.2,  # Increased shift range
                    shear_range=0.2,  # Increased shear range
                    zoom_range=0.2,  # Increased zoom range
                    horizontal_flip=True,
                    fill_mode="nearest",
                    validation_split=0.2
        )
    else:
        datagenTrain = ImageDataGenerator(
                    preprocessing_function=xception.preprocess_input,
                    validation_split=0.2
        )
        
    datagenVal = ImageDataGenerator(
                preprocessing_function=xception.preprocess_input,
                validation_split=0.2
    )   

        
    
    # Create a train and validation data generator
    train_gen = datagenTrain.flow_from_directory(
        input_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode="binary",
        subset="training",
        shuffle=True
    )

    val_gen = datagenVal.flow_from_directory(
        input_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode="binary",
        subset="validation",
        shuffle=True
    )
    
    return train_gen, val_gen

def create_test_dataset(
    input_dir,
    img_height=299,
    img_width=299,
    batch_size=32,
):
    """
    Creates a test dataset from images in the input directory.

    Args:
        input_dir (str): The path to the input directory containing 'real' and 'fake' subdirectories.
        img_height (int): The height of the input images. Default is 299.
        img_width (int): The width of the input images. Default is 299.
        batch_size (int): The batch size for the data generators. Default is 32.

    Returns:
        tf.data.Dataset: The test dataset.
    """
    # Create ImageDataGenerator
    datagen = ImageDataGenerator(
        preprocessing_function=xception.preprocess_input
    )

    return datagen.flow_from_directory(
        input_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode="binary",
        shuffle=True,
    )

# Experiments
---
## Experiment #0001 | Layer Configuration Exploration
**Experiment ID:** 0001  
**Experiment Description:** Initial testing to select a top_model layer configuration.
  
**Experiment Outcome:** 
**Next Test:** 


In [19]:
# Experiment ID:TMC-0001
experimentId = "TMC-0001"

# Load Directories (Leave alone unless specifying a different dataset)
experiment_train_directory = train_directory
experiment_test_directory = test_directory
experiment_directory = f"{experiment_base_directory}/{experimentId}"
experiment_results_directory = f"{experiment_directory}/results"
experiment_models_directory = f"{experiment_directory}/models"

# Create directories if they don't exist
os.makedirs(name=experiment_directory, exist_ok=True)
os.makedirs(name=experiment_models_directory, exist_ok=True)
os.makedirs(name=experiment_results_directory, exist_ok=True)

# Declare lists for report generation
accuracy_report = []
layers_report = []

# Define Early Stopping Callback
early_stopping = EarlyStopping(
    monitor='val_loss', 
    mode='min', 
    verbose=1, 
    patience=3
)

# Define Reduce Learning Rate on Plateau Callback
lr_reducer = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=2,
    verbose=1,
)

# Define Model Checkpoint Callback
model_checkpoint = ModelCheckpoint(f'{experiment_models_directory}/best_model.h5', 
monitor='val_loss', 
mode='min', 
verbose=1, 
save_best_only=True
)

# Create a dataset and preprocess images to suit base model
train_ds, val_ds = create_train_val_datasets(experiment_train_directory, batch_size=16)
test_ds = create_test_dataset(experiment_test_directory, batch_size=16)


# Create the base model from the pre-trained model (Xception)
base_model = Xception(
    weights="imagenet", 
    include_top=False,
    input_tensor=Input(shape=(299, 299, 3)))

# Freeze/Unfreeze the base model layers
base_model.trainable = True

# Add a classification head (Top Model)
top_model = base_model.output
top_model = GlobalAveragePooling2D()(top_model)
top_model = Dropout(0.5)(top_model)
predictions = Dense(1, activation='sigmoid')(top_model)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(
    optimizer=Adam(1e-3),  
    loss=BinaryCrossentropy(from_logits=False),
    metrics=[BinaryAccuracy(), Precision(), Recall()],
)


# Start Timer to measure processing time 
start_time = time.time()

# Train the model
history = model.fit(
    train_ds, 
    steps_per_epoch=train_ds.samples // train_ds.batch_size,
    epochs=50, 
    validation_data=val_ds,
    validation_steps=val_ds.samples // val_ds.batch_size,
    callbacks = [early_stopping, lr_reducer, model_checkpoint]
)


# End Timer and calculate processing time
end_time = time.time()
processing_time = end_time - start_time

# Evaluate the model
evaluate = model.evaluate(test_ds, steps = test_ds.samples // test_ds.batch_size, verbose=1)

# Reset the generator to make sure the order of predictions matches the order of labels
train_ds.reset() 
val_ds.reset()
test_ds.reset()

# Make predictions
train_preds = (model.predict(train_ds) > 0.5).astype("int32")
val_preds = (model.predict(val_ds) > 0.5).astype("int32")
test_preds = (model.predict(test_ds) > 0.5).astype("int32")

# Calculate additional metrics
train_precision = Precision()
train_recall = Recall()
train_f1 = 2 * ((precision * recall) / (precision + recall))

val_precision = precision(val_ds.classes, val_preds).numpy()
val_recall = recall(val_ds.classes, val_preds).numpy()
val_f1 = 2 * ((val_precision * val_recall) / (val_precision + val_recall + 1e-5))

# Compute confusion matrix for training set and test set
train_confusion_matrix = confusion_matrix(train_ds.classes, train_preds)
test_confusion_matrix = confusion_matrix(test_ds.classes, test_preds)

# Compute ROC curve for test set
test_fpr, test_tpr, _ = roc_curve(test_ds.classes, test_preds)
roc_auc = auc(test_fpr, test_tpr)

# Append results to accuracy_test list
accuracy_report.append({
    'processing_time': round(processing_time, 3),
    'test_accuracy': round(evaluate[1], 3),
    'test_loss': round(evaluate[0], 3),
    'test_precision': precision(test_ds.classes, test_preds).numpy(),
    'test_recall': recall(test_ds.classes, test_preds).numpy(),
    'test_f1': f1_score(test_ds.classes, test_preds),
    'test_confusion_matrix': test_confusion_matrix,
    'test_roc_curve': (test_fpr, test_tpr),
    'test_roc_auc': roc_auc,
    'train_accuracy': round(history.history['binary_accuracy'][-1],3),
    'train_loss': round(history.history['loss'][-1],3),
    'train_precision': train_precision(train_ds.classes, train_preds).numpy(),
    'train_recall': train_recall(train_ds.classes, train_preds).numpy(),
    'train_f1': train_f1.numpy(),
    'train_confusion_matrix': train_confusion_matrix,
    'val_accuracy': round(history.history['val_binary_accuracy'][-1],3),
    'val_loss': round(history.history['val_loss'][-1],3),
    'val_precision': val_precision,
    'val_recall': val_recall,
    'val_f1': val_f1,
})

for layer in base_model.layers:
    layers.append({
        'layer_name': layer.name,
        'layer_trainable': layer.trainable
    })
    

# Create DataFrame with accuracy report & layers report
accuracy_report_results = pd.DataFrame(accuracy_test)
layers_report_results = pd.DataFrame(layers)

# Save accuracy report & layers report to CSV
accuracy_report_results.to_csv(f'{experiment_results_directory}/accuracy_report_results.csv', index=False)
layers_report_results.to_csv(f'{experiment_results_directory}/layers_report_results.csv', index=False)

# Create and save confusion matrix plots
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
sns.heatmap(train_confusion_matrix, annot=True, fmt="d", ax=ax[0])
ax[0].set_title("Train Confusion Matrix")
sns.heatmap(test_confusion_matrix, annot=True, fmt="d", ax=ax[1])
ax[1].set_title("Test Confusion Matrix")
plt.savefig(f'{experiment_results_directory}/confusion_matrix.png')

# Create and save ROC curve plot
plt.figure(figsize=(8, 8))
plt.plot(test_fpr, test_tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig(f'{experiment_results_directory}/roc_curve.png')

Found 9660 images belonging to 2 classes.
Found 2414 images belonging to 2 classes.
Found 2139 images belonging to 2 classes.


2023-05-24 16:21:54.177043: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 6.00MiB (rounded to 6291456)requested by op Mul
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-05-24 16:21:54.177086: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-05-24 16:21:54.177099: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 56, Chunks in use: 56. 14.0KiB allocated for chunks. 14.0KiB in use in bin. 3.2KiB client-requested in use in bin.
2023-05-24 16:21:54.177107: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 24, Chunks in use: 24. 12.8KiB allocated for chunks. 12.8KiB in use in bin. 12.0KiB client-requested in use in bin.
2023-05-24 16:21:54.177123: I tensorflow/tsl/framework/bfc_al

ResourceExhaustedError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Mul]